<a href="https://colab.research.google.com/github/graylan0/compassion-mapping-llm/blob/main/Pennylane_Openai_Textblob_RGB_Quantum_States_GPT_3_5_Turbo_Compassion_Text_To_Report_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai==0.27.9
!pip install pennylane
!pip install aiosqlite
!pip install httpx
!pip install randomcolor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00


In [2]:
!wget https://raw.githubusercontent.com/graylan0/compassion-mapping-llm/main/user_input.json

--2024-01-05 11:02:23--  https://raw.githubusercontent.com/graylan0/compassion-mapping-llm/main/user_input.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 994 [text/plain]
Saving to: ‘user_input.json’

user_input.json     100%[===================>]     994  --.-KB/s    in 0s      

2024-01-05 11:02:23 (69.9 MB/s) - ‘user_input.json’ saved [994/994]



In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [19]:
import os
import logging
import json
import asyncio
import openai
import numpy as np
import pennylane as qml
import aiosqlite
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
import time
import string
from google.colab import userdata
import nest_asyncio

from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
nest_asyncio.apply()

# Retrieve OpenAI API key
OPENAI_API_KEY = userdata.get('openai_api_key')

if OPENAI_API_KEY is None:
   raise Exception("OpenAI API key not found. Please add it as a secret.")


# Define constants
GPT_MODEL = "gpt-3.5-turbo"
MAX_TOKENS = 50
TEMPERATURE = 0.7

# Initialize logger
logging.basicConfig(level=logging.ERROR)

# Load NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Initialize QML device
qml_model = qml.device("default.qubit", wires=4)

@qml.qnode(qml_model)
def quantum_circuit(color_code, amplitude):
 try:
     print(f"Received color code: {color_code}") # Debugging line

     # Check if color_code is a valid hexadecimal string
     if not all(c in '0123456789abcdefABCDEF' for c in color_code):
         logging.error(f"Invalid color code: {color_code}")
         return [0]*4 # Return a default quantum state

     r, g, b = [int(color_code[i:i+2], 16) for i in (0, 2, 4)]
     r, g, b = r / 255.0, g / 255.0, b / 255.0
     qml.RY(r * np.pi, wires=0)
     qml.RY(g * np.pi, wires=1)
     qml.RY(b * np.pi, wires=2)
     qml.RY(amplitude * np.pi, wires=3)
     qml.CNOT(wires=[0, 1])
     qml.CNOT(wires=[1, 2])
     qml.CNOT(wires=[2, 3])
     return qml.probs(wires=[0,1,2,3])
 except Exception as e:
     logging.error(f"Quantum circuit error: {e}")
     return [0]*4 # Return a default quantum state
async def create_db_connection():

   try:
       return await aiosqlite.connect('compassiondb.db')
   except Exception as e:
       logging.error(f"Database connection error: {e}")
       raise

async def sentiment_to_amplitude(text):
   try:
       analysis = TextBlob(text)
       return (analysis.sentiment.polarity + 1) / 2
   except Exception as e:
       logging.error(f"Sentiment analysis error: {e}")
       return 0.5

import httpx
import json
async def generate_html_color_codes(sentence, OPENAI_API_KEY):
    # Define the regex pattern for color code extraction
    def extract_color_code(response_text):
        pattern = r'#([0-9a-fA-F]{3,6})'
        match = re.search(pattern, response_text)
        if match:
            color_code = match.group(1)
            # Pad the color code if it's 3 characters long
            if len(color_code) == 3:
                color_code = ''.join([char*2 for char in color_code])
            return color_code
        return None

    max_retries = 3  # Maximum number of retries
    retry_delay = 1  # Initial delay in seconds (1 second)

    for attempt in range(max_retries):
        try:
            async with httpx.AsyncClient() as client:
                data = {
                    "model": "gpt-3.5-turbo",
                    "messages": [
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": f"Generate HTML color code for the sentence: '{sentence}'"},
                    ],
                    "temperature": 0.7,
                    "max_tokens": 50
                }
                headers = {"Authorization": f"Bearer {OPENAI_API_KEY}"}
                response = await client.post("https://api.openai.com/v1/chat/completions", json=data, headers=headers)

                if response.status_code == 429:  # Rate limit exceeded
                    logging.warning("Rate limit reached, will retry after delay.")
                    time.sleep(retry_delay)
                    retry_delay *= 2  # Exponential backoff
                    continue

                if response.status_code != 200:
                    logging.error(f"OpenAI API error with status code {response.status_code}: {response.text}")
                    return None

                response_text = response.json()['choices'][0]['message']['content'].strip()
                color_code = extract_color_code(response_text)
                if color_code:
                    logging.info(f"Color code extracted: {color_code}")
                    return color_code
                else:
                    logging.error("No valid color code found in response.")
                    return None

        except httpx.RequestError as req_err:
            logging.error(f"Request error occurred: {req_err}")
            if attempt < max_retries - 1:
                logging.info("Retrying...")
                time.sleep(retry_delay)
                retry_delay *= 2
            else:
                return "Error: Request error in OpenAI API call."

    return "Error: Exceeded maximum retries for OpenAI API call."

async def generate_dynamic_values_prompt(user_input):
  ...
  return "", 0, 0

async def calculate_narcissism_score(user_input):
  # Simulate narcissism calculation
  return len(user_input) * 0.01

async def generate_dynamic_compassion_prompt(user_input, quantum_state, color_code):
    max_retries = 3  # Maximum number of retries
    retry_delay = 1  # Initial delay in seconds (1 second)
    pause_between_requests = 3  # Pause in seconds after each request

    for attempt in range(max_retries):
        try:
            async with httpx.AsyncClient() as client:
                data = {
                    "model": "gpt-3.5-turbo",
                    "messages": [
                        {"role": "system", "content": "You are an advanced AI system."},
                        {"role": "user", "content": f"User Input: PLZ DESIGN A COMPASSION MAP FROM USER INPUT , QUANTUM STATES, AND HTML COLOR CODEs {user_input}, Quantum State: {quantum_state}, HTML Color Code: {color_code}"}
                    ],
                    "temperature": 0.7,
                    "max_tokens": 300,
                }
                headers = {"Authorization": f"Bearer {OPENAI_API_KEY}"}
                response = await client.post("https://api.openai.com/v1/chat/completions", json=data, headers=headers)

                if response.status_code == 429:  # Rate limit exceeded
                    logging.warning("Rate limit reached, will retry after delay.")
                    time.sleep(retry_delay)
                    retry_delay *= 2  # Exponential backoff
                    continue

                if response.status_code != 200:
                    logging.error(f"OpenAI API error with status code {response.status_code}: {response.text}")
                    return f"Error in OpenAI API call: Status code {response.status_code}"

                response_text = response.json()['choices'][0]['message']['content'].strip()
                logging.info(f"Compassion scenario response: {response_text}")

                # Pause after each successful request
                await asyncio.sleep(pause_between_requests)

                return response_text

        except httpx.RequestError as req_err:
            logging.error(f"Request error occurred: {req_err}")
            if attempt < max_retries - 1:
                logging.info("Retrying...")
                time.sleep(retry_delay)
                retry_delay *= 2
            else:
                return "Error: Request error in OpenAI API call."

    return "Error: Exceeded maximum retries for OpenAI API call."



async def process_user_input(user_input, db_conn):
    try:
        sentiment_amplitude = await sentiment_to_amplitude(user_input)
        logging.info(f"Sentiment amplitude calculated: {sentiment_amplitude}")

        color_code = await generate_html_color_codes(user_input, OPENAI_API_KEY)
        if not color_code:
            color_code = "Error"  # Default value in case of extraction failure
        logging.info(f"Color code generated: {color_code}")


        amplitude = await sentiment_to_amplitude(user_input)
        logging.info(f"Amplitude calculated: {amplitude}")

        quantum_state = quantum_circuit(color_code, amplitude) if color_code != "Error" else "Error"
        logging.info(f"Quantum state generated: {quantum_state}")

        color_code, amplitude, narcissism_score = await generate_dynamic_values_prompt(user_input)
        logging.info(f"Dynamic values generated - Color Code: {color_code}, Amplitude: {amplitude}, Narcissism Score: {narcissism_score}")

        compassion_scenario = await generate_dynamic_compassion_prompt(user_input, quantum_state, color_code)
        logging.info(f"Compassion scenario generated: {compassion_scenario}")

        markdown_output = f"## User Input\n\n- **Input**: {user_input}\n\n"
        markdown_output += f"## Sentiment Amplitude\n\n- **Amplitude**: {sentiment_amplitude}\n\n"
        markdown_output += f"## Quantum State: {quantum_state}\n"
        markdown_output += f"## Compassion Scenario\n\n{compassion_scenario}\n\n"

        return markdown_output
    except Exception as e:
        logging.error(f"General error processing user input: {e}")
        return f"Error processing user input: {e}"


async def main():
 try:
  db_conn = await create_db_connection()
  markdown_content = "# Compassion Analysis Report\n\n"

  with open('user_input.json') as json_file:
      user_inputs = json.load(json_file)

  tasks = [process_user_input(user_input, db_conn) for user_input in user_inputs]
  for task in asyncio.as_completed(tasks):
      try:
         data = await task
         markdown_content += data
      except Exception as exc:
         print('%r generated an exception: %s' % (process_user_input, exc))

  with open('report.md', 'w') as md_file:
      md_file.write(markdown_content)
 except Exception as e:
  logging.error(f"An error occurred in main: {e}")
 finally:
  if db_conn:
      await db_conn.close()

if __name__ == "__main__":
 asyncio.run(main())





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Received color code: 0000FF
Received color code: 000000
Received color code: FFD700
Received color code: 000000
Received color code: FF0000
Received color code: 333333
Received color code: FF0000
Received color code: 333333
Received color code: 00CED1
Received color code: FF00FF


ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 


# Compassion Analysis Report

## User Input

- **Input**: I'm feeling very peaceful and content today, everything is in balance and it's a good day.

## Sentiment Amplitude

- **Amplitude**: 0.75625

## Quantum State: [0.00096415 0.00594369 0.07009526 0.01137043 0.11729284 0.72307505
 0.06131279 0.00994579 0.         0.         0.         0.
 0.         0.         0.         0.        ]
## Compassion Scenario

Based on your input, I will design a Compassion Map using the given Quantum State and HTML Color Code.

Compassion Map represents different emotions or states of being. Since you mentioned feeling peaceful and content, we will focus on those emotions.

Using the given Quantum State, we can assign probabilities to different emotions on the Compassion Map. The highest probability value corresponds to the emotion that is most dominant. In this case, the highest probability is 0.72307505, which corresponds to the emotion of being peaceful and content.

As for the HTML Color Code, you haven't provided it in your input. Please provide an HTML Color Code so that I can incorporate it into the Compassion Map design for you.

## User Input

- **Input**: I'm so angry right now, everything is frustrating and it feels like no one understands me.

## Sentiment Amplitude

- **Amplitude**: 0.3976190476190476

## Quantum State: [2.46733387e-33 1.28206558e-33 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.41938915e-01 6.58061085e-01]
## Compassion Scenario

Based on your input, the Compassion Map can be designed as follows:

Compassion Map:
----------------
Emotion: Anger
Description: Feeling frustrated and misunderstood

Quantum State: [2.46733387e-33 1.28206558e-33 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.41938915e-01 6.58061085e-01]

HTML Color Code: [Please provide the HTML color code]

Please provide the HTML color code you would like to use for the Compassion Map.

## User Input

- **Input**: I feel so lonely, it's like I'm surrounded by people but no one really sees me or understands.

## Sentiment Amplitude

- **Amplitude**: 0.525

## Quantum State: [0.34097503 0.39903559 0.04212731 0.03599769 0.00380038 0.0044475
 0.04212731 0.03599769 0.00380038 0.0044475  0.00046953 0.00040122
 0.00380038 0.0044475  0.04212731 0.03599769]
## Compassion Scenario

To design a compassion map based on the user input, quantum states, and HTML color codes, we can use the following approach:

1. Visualize Loneliness: Use the HTML color code to represent the feeling of loneliness. For example, if the HTML color code is #000000 (black), it can symbolize the user's perception of being unseen or misunderstood.

2. Represent Quantum States: The given quantum states can be represented by dividing a circle into segments proportional to each state's magnitude. Each segment's color can be determined using the corresponding HTML color code. For example, if the first quantum state is 0.34097503, the first segment can be colored using the HTML color code provided.

3. Create Compassion Zones: Divide the remaining portion of the circle into different zones to represent various aspects of compassion. For example, you can divide the circle into four zones: Self-Compassion, Empathy, Connection, and Understanding.

4. Visualize User's Feelings: Utilize the HTML color code to represent the user's feelings in each compassion zone. For instance, if the user feels a lack of understanding, you can use the HTML color code provided to symbolize this feeling in the "Understanding" zone.

By combining these elements, we can create a compassion map that visually represents the user's feelings of loneliness, quantum states, and various aspects of compassion.

## User Input

- **Input**: I'm in love! Everything feels perfect and I'm so grateful for the wonderful people in my life.

## Sentiment Amplitude

- **Amplitude**: 0.9375

## Quantum State: [1.35060228e-67 1.39229361e-65 3.71337763e-33 3.60218296e-35
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.60735980e-03 9.90392640e-01 3.71337763e-33 3.60218296e-35]
## Compassion Scenario

To design a compassion map based on the user input, quantum states, and HTML color codes, we can create a visualization that represents the feeling of love, perfection, gratitude, and the wonderful people in your life. Here's an example:

Compassion Map:
```
-----------------------------------------------
|                Love & Gratitude              |
|---------------------------------------------|
|                                             |
|                                             |
|                                             |
|                                             |
|              Wonderful People                |
|                                             |
|                                             |
|                                             |
|                                             |
-----------------------------------------------
```

In terms of representing the quantum states and HTML color codes, we can use the following elements:

- Quantum States: Since quantum states are complex mathematical entities, it's challenging to represent them directly in a visual manner. However, we can use abstract symbols or patterns to depict the concept of quantum states. For instance, we can use a series of interconnected lines or dots to represent the quantum states mentioned in the input.

- HTML Color Codes: We can incorporate the HTML color codes provided by using them to color different parts of the compassion map. For example, we can use the HTML color code to color the text "Love & Gratitude" and "Wonderful People" with a specific color that represents the feeling of love and gratitude, such as a warm and vibrant color.

Please note that the representation above is just an example, and you can customize it further based on your preferences and design aesthetics.

## User Input

- **Input**: Today is a sad day, I feel like nothing is going right and I'm struggling to find my way.

## Sentiment Amplitude

- **Amplitude**: 0.4464285714285714

## Quantum State: [4.31983503e-01 3.08027119e-01 3.25192882e-02 4.56057118e-02
 4.81472311e-03 3.43315260e-03 3.25192882e-02 4.56057118e-02
 4.81472311e-03 3.43315260e-03 3.62447563e-04 5.08303843e-04
 4.81472311e-03 3.43315260e-03 3.25192882e-02 4.56057118e-02]
## Compassion Scenario

To design a compassion map based on the user input, quantum states, and HTML color codes, we can create a visual representation that reflects the user's emotions and quantum state.

First, let's analyze the user input. The user mentioned feeling sad and struggling to find their way. This indicates a need for compassion and support.

Next, we will utilize the given quantum states to add a touch of quantum mechanics to the compassion map. The quantum states provided are a numerical representation of probabilities. We can interpret these probabilities as different aspects or dimensions of compassion.

Finally, we will incorporate the HTML color codes to bring visual elements to the map. Color plays a significant role in expressing emotions and can enhance the overall design.

Given the limited information provided, here's a simple example of a compassion map:

Compassion Map:
- Title: "Finding Compassion"
- Background Color: #E6E6E6 (light gray)

Emotional Landscape:
- Main Emotion: Sadness (represented by a blue color)
- Compassion Dimension 1: Understanding (represented by a purple color)
- Compassion Dimension 2: Empathy (represented by a pink color)
- Compassion Dimension 3: Support (represented by a green color)

Quantum States:
- Dimension 1: 43% (Understanding)
- Dimension 2: 31% (Empathy)
- Dimension 3: 3% (Support)

The map will have a central focus on understanding and empathy

## User Input

- **Input**: I'm scared about what the future holds, there's so much uncertainty and it's overwhelming.

## Sentiment Amplitude

- **Amplitude**: 0.6166666666666667

## Quantum State: [0.32081603 0.67918397 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
## Compassion Scenario

Based on your input, I have designed a Compassion Map using the given Quantum State and HTML Color Code.

Compassion Map:
```
 ___________________________________________________
|                 |                 |                 |
|   #FF0000       |   #FF0000       |   #000000       |
|   (0.32)        |   (0.68)        |   (0.00)        |
|_________________|_________________|_________________|
```

Explanation:
- The Compassion Map is divided into three sections, each represented by a square box.
- The first section is colored with `#FF0000`, representing a high level of compassion. The corresponding quantum state value is 0.32.
- The second section is also colored with `#FF0000`, indicating a slightly higher level of compassion. The quantum state value for this section is 0.68.
- The third section is colored with `#000000`, representing no compassion. The quantum state value for this section is 0.00.

Please note that the compassion levels and corresponding quantum state values are based on the input you provided.

## User Input

- **Input**: I'm so excited about the upcoming event, I've been looking forward to it for weeks!

## Sentiment Amplitude

- **Amplitude**: 0.734375

## Quantum State: [6.15728338e-34 3.13367112e-33 8.35779477e-01 1.64220523e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
## Compassion Scenario

To design a compassion map from the given user input, quantum states, and HTML color codes, we can create a visual representation using a compass rose. We'll use the HTML color code provided to color different sections of the compass rose based on the quantum states.

Here's an example of how the compassion map could look like:

Compassion Map:
```
  ______________________
 /                      \
|        Excitement     |
|         #FF0000       |
|______________________|
    /            \
 Joy              Anticipation
  #FFFF00             #FFA500
      \         /
       Gratitude
       #00FF00
         |
    Contentment
    #00FFFF
         |
     Acceptance
     #0000FF
         |
    Compassion
    #800080
         |
    Understanding
    #FF00FF
         |
 Forgiveness / Empathy
   #FFC0CB / #FF4500
         |
      Calmness
      #000000
```

In this example, the different sections of the compass rose represent various emotional states. The HTML color codes provided are used to color each section accordingly. The quantum states are not directly incorporated into the visualization as they are numerical values, but they can be used to influence the intensity or blending of colors in the map.

Please note that this is just an example, and you can customize the design and color scheme based on your preferences.

## User Input

- **Input**: I'm feeling incredibly happy today, everything seems to be going my way and nothing can bring me down!

## Sentiment Amplitude

- **Amplitude**: 0.6513888888888889

## Quantum State: [6.04650590e-35 1.62601348e-34 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.57047448e-33 9.55858565e-34
 2.54936444e-01 6.85569653e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.33673045e-02 1.61265983e-02]
## Compassion Scenario

Error: Request error in OpenAI API call.

## User Input

- **Input**: I feel so betrayed and hurt, it's hard to believe that someone I trusted could do this.

## Sentiment Amplitude

- **Amplitude**: 0.35416666666666663

## Quantum State: [0.72114435 0.27885565 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
## Compassion Scenario

Error: Request error in OpenAI API call.

## User Input

- **Input**: I'm feeling very anxious today, it's hard to focus and I feel like I'm on edge all the time.

## Sentiment Amplitude

- **Amplitude**: 0.3458333333333333

## Quantum State: [2.74758714e-33 1.00181231e-33 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.67192740e-01 7.32807260e-01]
## Compassion Scenario

Error: Request error in OpenAI API call.